In [ ]:
# Coded by G. Petricca (@gmrpetricca)

from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read_inventory
from math import sin, cos, sqrt, atan2, radians
import urllib.request
import pandas as pd
import numpy as np
import time
import os

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# read raw data from excel file and create a dataframe (data from http://www.fdsn.org/networks/detail/AM/)
# Raspberry Shake stations refined for the W portion of the USA, in a box approx. 32<lat<49 N and 114<lon<124 W
dfs = pd.read_excel('ShakeNetwork2020.xlsx', sheet_name="W USA Shake Network 2020")

dfs_quake = pd.read_excel('ShakeNetwork2020.xlsx', sheet_name="Earthquake List")

dfs_events = dfs_quake['UTC Date Time'].tolist()

for j in range(0, len(dfs_events)):
    
    # distance from earthquake calculations using Haversine Formula
    eqdist = []

    lat1 = radians(dfs_quake['Latitude'][j])
    lon1 = radians(dfs_quake['Longitude'][j])
    
    for k in range(0, len(dfs)):

        lat2 = radians(dfs['Latitude'][k])
        lon2 = radians(dfs['Longitude'][k])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        R = 6371.0 # approximate radius of earth in km

        distance = R * c

        eqdist.append(round(distance,1)) # distance from Earthquake in km
        #end distance from earthquake calculations

    # add distance from event column
    dfs['Distance'] = eqdist

    # ask the user for a distance limit and select only the appropriate stations
    while True:
        try:
            limit = int(input("\n" + "Please insert a km radius to select only the relevant stations. Leave empty to use all.")) 
        except ValueError:
            limit = 20000 # dummy value to include every station in the dataframe

        dfs_def = dfs.loc[(dfs['Distance'] <= limit)]

        if dfs_def.empty == False:
            # produce and print out some statistics
            stations_number = len(dfs_def['Distance']) 

            if limit == 20000: 
                print("\n" + "There are " + str(stations_number) + " stations whitin the max input range." + "\n")
            else:
                print("\n" + "There are " + str(stations_number) + " stations whitin the " + str(limit) + " km input range." + "\n")
            break

        print("\n" + "The Station Count is equal to zero. Please input a greater distance.")
    
    # define fdsn client to get data from
    # this is just to verify if a station was online or offline near the time of the event
    event_time = dfs_events[j]
    
    client = Client('http://fdsnws.raspberryshakedata.com')
    orig_time = UTCDateTime(event_time)
    t1 = orig_time - 15
    t2 = orig_time + 15

    dfs_stations = dfs_def['Station Code'].tolist()
    
    # parse inventories for each station to list their channels
    sta_chan = []
    for retry in range (0, len(dfs_stations)):
        try:
            STATION = dfs_stations[retry]
            inv = read_inventory('https://fdsnws.raspberryshakedata.com/fdsnws/station/1/query?network=AM&station=%s&level=resp&format=xml&nodata=404&starttime=%s' % (STATION, str(orig_time)))
            net = inv[0]
            sta = net[0]
            cha = len(sta)
            
            # append the appropriate channels for each online station
            if cha == 4: 
                sta_chan.append("EHZ - ENZ - ENN - ENE")
            elif cha == 3:
                sta_chan.append("EHZ - EHN - EHE")
            else:
                if "SHZ" in str(sta):
                    sta_chan.append("SHZ")
                else: 
                    sta_chan.append("EHZ")
                    
        except Exception as e:
            #print(e) # in case of errors
            sta_chan.append(' - ')
            pass     
    
    # create an automatic selector to append the online/offline status of each station in the network
    onsta = []
    for retry in range (0, len(dfs_stations)):
        try:
            STATION = dfs_stations[retry]
            st = client.get_waveforms("AM", STATION, "00", "*HZ", starttime=t1, endtime=t2, attach_response=True)
            str_st = str(st)
            if str_st: 
                onsta.append('Yes')
            else:
                onsta.append('No')
                pass
        except Exception as e:
            #print(e) # in case of errors
            onsta.append('No')
            pass
        
    # put everything into a final dataframe
    dfs_def['Online'] = onsta
    dfs_def['Channels'] = sta_chan
    
    # produce and print out some statistics
    total_stations = len(dfs_def['Online'])
    online_stations = dfs_def['Online'].str.count("Yes").sum() 
    
    # prepare name of final .csv for each event
    file_name = str(event_time).replace('-','').replace(':','').replace('.', '').replace('T', '')
    
    print("For the event @ " + str(event_time) + " there were " + str(online_stations) + " online stations out of " + str(total_stations) + " total stations." + "\n")
    
    # create a folder for each earthquake in the list
    path = file_name
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    # save the online stations dataframe to .csv file
    dfs_def.to_csv(os.path.join(path,r'Online Stations - ' + file_name + '.csv'))
    
    # prepare conditions for the mSEED files download
    dfs_down = dfs_def.loc[(dfs_def['Online'] != "No")]

    dfs_shakes = dfs_down['Station Code'].tolist()
    
    start_time = orig_time - 300
    end_time = orig_time + 300
    
    # download the mSEED files for each available channel of each online station in the range
    for count in range(0, len(dfs_down)):
        
        station = dfs_shakes[count]
        
        channels = ["SHZ","EHZ","EHN","EHE","ENZ","ENN","ENE"]
        
        for chan in range(0, len(channels)):
            channel = channels[chan]
            url = "https://fdsnws.raspberryshakedata.com/fdsnws/dataselect/1/query?net=AM&sta=" + str(station) + "&loc=00&cha=" + str(channel) + "&start=" + str(start_time) + "&end=" + str(end_time) + ""
            urllib.request.urlretrieve(url, os.path.join(path, r'' + file_name + '.AM.' + station + '.' + channel + '.mseed'))
            
    # delete empty files, if there are any
    str_directory = path
    
    list_files = [x for x in os.listdir(str_directory) if x[0]!='.']
    
    for each_file in list_files:
        file_path = '%s/%s' % (str_directory, each_file)
        
        if os.path.getsize(file_path)==0:
            os.remove(file_path)
        else:
            pass
    
    # conclude program
    print("Download mSEED Files Complete." + "\n")